# Welcome to the Pneumonia Image Classifier
## In this notebook we're going to train a Neural Network to detect pneumonia in images.

### Let's get started by importing our libraries

In [1]:
from __future__ import print_function
import keras

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import math


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Let's create a reference to our data directories

In [2]:
train_data_dir = './input/train'
test_data_dir = './input/test'
val_data_dir = './input/val'

### Define some variables that we're going to use in training and testing

In [3]:
nb_train_samples = 5217
nb_validation_samples = 17
epochs = 20
batch_size = 16

### Shape our input

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

input_shape = (img_width, img_height, 3)
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)

### Given the size of our dataset, we'll need to use generators in order to feed the data to our model bit-by-bit
### Let's create our generators

In [5]:
train_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [6]:
test_gen = ImageDataGenerator(rescale=1. / 255)

In [7]:
train_generator = train_gen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [8]:
test_generator = test_gen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [9]:
validation_generator = test_gen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 16 images belonging to 2 classes.


### Now let's initialize our model

In [10]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(48, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

### Compile it

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Fit it using our training generator... it's going to take a while

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
326/326 [==============================] - 187s 574ms/step - loss: 0.4679 - acc: 0.7847 - val_loss: 0.8095 - val_acc: 0.7500
Epoch 2/20
326/326 [==============================] - 182s 559ms/step - loss: 0.3106 - acc: 0.8719 - val_loss: 0.5392 - val_acc: 0.8750
Epoch 3/20
326/326 [==============================] - 200s 614ms/step - loss: 0.2719 - acc: 0.9022 - val_loss: 0.6166 - val_acc: 0.8125
Epoch 4/20
326/326 [==============================] - 201s 618ms/step - loss: 0.2355 - acc: 0.9122 - val_loss: 0.8213 - val_acc: 0.7500
Epoch 5/20
326/326 [==============================] - 202s 621ms/step - loss: 0.2337 - acc: 0.9174 - val_loss: 0.6801 - val_acc: 0.7500
Epoch 6/20
326/326 [==============================] - 194s 594ms/step - loss: 0.2257 - acc: 0.9231 - val_loss: 2.3351 - val_acc: 0.6250
Epoch 7/20
326/326 [==============================] - 199s 612ms/step - loss: 0.2233 - acc: 0.9268 - val_loss: 0.8405 - val_acc: 0.7500
Epoch 8/20
326/326 [============================

### Now let's evaluate it using our test generator

In [13]:
score = model.evaluate_generator(test_generator, math.ceil(5217 / batch_size), verbose=1)

327/327 [==============================] - 81s 248ms/step


In [14]:
score

[0.47056583494376264, 0.9030963302752294]